In [1]:
from IPython.display import display

import numpy as np
import pandas as pd

from conf.settings import (
    get_settings, Settings,
    get_read_file_settings, ReadFileSettings,
)
from recsys_tasks.processors.users_based import UserBasedRecommendationsProcessor


In [2]:
# Получение конфигурации для расчетов.
settings: Settings = get_settings()
read_file_settings: ReadFileSettings = get_read_file_settings()

In [3]:
# Получение матричного представления исходных данных.
input_data_filename = '4_users_based.csv'
input_data_filepath = settings.INPUT_DATA_DIR / input_data_filename

source_data: np.ndarray = np.genfromtxt(
    input_data_filepath,
    dtype=float,
    **read_file_settings.model_dump(),
)
source_data: np.ndarray[tuple[int, int], int]
pd.DataFrame(source_data)


,0,1,2,3,4,5,6,7,8,9,10
0,4.0,3.0,2.0,2.0,4.0,5.0,1.0,1.0,4.0,1.0,NaN
1,3.0,4.0,3.0,5.0,3.0,4.0,2.0,1.0,NaN,2.0,NaN
2,5.0,5.0,2.0,4.0,4.0,3.0,NaN,NaN,NaN,1.0,NaN
3,4.0,4.0,3.0,3.0,1.0,4.0,1.0,3.0,3.0,2.0,NaN
4,4.0,5.0,3.0,2.0,2.0,2.0,2.0,3.0,4.0,3.0,NaN


In [4]:
def display_recommendation_results(rec_processor: UserBasedRecommendationsProcessor):
    """Отображение результатов работы процессора."""
    results = rec_processor()

    print('Не оцененные пользователем продукты:', rec_processor.not_rated_products)
    
    if not rec_processor.is_new_user:
        print(
            'Минимальное пороговое значение для рекомендаций (средний рейтинг пользователя):',
            rec_processor.mean_user_score,
        )
    
    print()

    print('Рекомендованные продукты:')
    for recommended_product in results.recommended_products:
        print(
            f'Продукт №{recommended_product.product_id}: '
            f'рассчитанный рейтинг – {recommended_product.calculated_score}.'
        )

    print()

    print('Данные продукты не попадут в рекомендации:')
    for not_recommended_product in results.not_recommended_products or []:
        print(
            f'Продукт №{not_recommended_product.product_id}: '
            f'рассчитанный рейтинг – {not_recommended_product.calculated_score}.'
        )
    
    if rec_processor.is_new_user:
        print('\nДанный расчет был проведен для нового пользователя.')
        return

    print('\nМатрица предпочтений после исключения пользователей, не оценивших те же продукты:')
    displayed_matrix = rec_processor.user_preferences_matrix_for_similarity
    display(
        pd.DataFrame(
            displayed_matrix.matrix,
            columns=displayed_matrix.columns,
            index=displayed_matrix.index,
        )
    )
    
    print('Матрица косинусного подобия пользователей:')
    displayed_matrix = rec_processor.similarity_matrix_of_users_rated
    display(
        pd.DataFrame(
            displayed_matrix.matrix,
            columns=displayed_matrix.columns,
            index=displayed_matrix.index,
        )
    )
    
    print('Оценки похожих пользователей:')
    displayed_matrix = rec_processor.similar_rated_users_preferences_matrix
    display(
        pd.DataFrame(
            displayed_matrix.matrix,
            columns=displayed_matrix.columns,
            index=displayed_matrix.index,
        )
    )


In [5]:
# Для существующего пользователя.
processor = UserBasedRecommendationsProcessor(source_data, user_id=8, min_similarity_coefficient=0.95)
display_recommendation_results(processor)


Не оцененные пользователем продукты: [1, 2]
Минимальное пороговое значение для рекомендаций (средний рейтинг пользователя): 3.6666666666666665

Рекомендованные продукты:
Продукт №2: рассчитанный рейтинг – 4.011076387454255.

Данные продукты не попадут в рекомендации:
Продукт №1: рассчитанный рейтинг – 3.3317588437747805.

Матрица предпочтений после исключения пользователей, не оценивших те же продукты:


,0,1,2,3,4,5,8,9
0,4.0,3.0,2.0,2.0,4.0,5.0,4.0,1.0
1,3.0,4.0,3.0,5.0,3.0,4.0,0.0,2.0
2,5.0,5.0,2.0,4.0,4.0,3.0,0.0,1.0
3,4.0,4.0,3.0,3.0,1.0,4.0,3.0,2.0
4,4.0,5.0,3.0,2.0,2.0,2.0,4.0,3.0


Матрица косинусного подобия пользователей:


,0,1,2,3,4,5,8,9
0,NaN,0.979796,0.984732,0.980196,0.881917,0.946729,0.991837,0.925820
1,0.979796,NaN,0.994987,0.960392,0.802377,0.864356,0.971797,0.982708
2,0.984732,0.994987,NaN,0.982467,0.790912,0.889898,0.965594,0.968665
3,0.980196,0.960392,0.982467,NaN,0.793884,0.940032,0.946943,0.907485
4,0.881917,0.802377,0.790912,0.793884,NaN,0.910840,0.920158,0.699854
5,0.946729,0.864356,0.889898,0.940032,0.910840,NaN,0.931240,0.756978
8,0.991837,0.971797,0.965594,0.946943,0.920158,0.931240,NaN,0.918262
9,0.925820,0.982708,0.968665,0.907485,0.699854,0.756978,0.918262,NaN


Оценки похожих пользователей:


,0,1,2
0,4.0,3.0,2.0
1,3.0,4.0,3.0
2,5.0,5.0,2.0
3,4.0,4.0,3.0
4,4.0,5.0,3.0


In [6]:
# Для нового пользователя.
processor = UserBasedRecommendationsProcessor(source_data, user_id=10)
display_recommendation_results(processor)

Не оцененные пользователем продукты: [0, 1, 2, 3, 4]

Рекомендованные продукты:
Продукт №2: рассчитанный рейтинг – 3.4285714285714284.

Данные продукты не попадут в рекомендации:

Данный расчет был проведен для нового пользователя.
